In [1]:
import pandas as pd
import pickle
import os
import glob
import time
import numpy as np

In [2]:
features = ['predict_' + str(i) for i in range(1, 11)] 
for day in range(6, 10):
    df = pd.read_csv('../dataset/rainfall_data_day' + str(day) + '.csv')
    df['mean'] = df[features].mean(axis=1)
    for i in range(3, 21):
        day_hour = df[df['hour'] == i]
        df_real_day = day_hour.copy()
        xid = df_real_day[df_real_day['hour'] == i]['xid']
        yid = df_real_day[df_real_day['hour'] == i]['yid'] 
        rainfall = df_real_day[df_real_day['hour'] == i]['mean']
        df_test_hour = pd.DataFrame({'xid': list(xid),
                      'yid': list(yid),
                      'rainfall': list(rainfall)})
        pt = df_test_hour.pivot_table(index='xid', columns='yid', values='rainfall', aggfunc=np.sum)
        store_dir = '../dataset/'
        if not os.path.exists(store_dir):
            os.mkdir(store_dir)
        with open(os.path.join(store_dir, 'rainfall_day' + str(day) + 'hour'+ str(i) +'.pickle'), 'wb') as f:
            pickle.dump(pt.values, f)

In [3]:
for day in range(6, 10):
    dirpath ='../dataset/'
    wind_matrix = -1
    for hour in range(3, 21, 1):
        filename = 'rainfall_day'+ str(day) +'hour' + str(hour) + '.pickle'
        with open(os.path.join(dirpath, filename), 'rb') as f:
            matrix = np.array(pickle.load(f, encoding='latin1'))

        if isinstance(wind_matrix, int):
            wind_matrix = matrix[:, :, np.newaxis]
        else:
            wind_matrix = np.concatenate([wind_matrix, matrix[:, :, np.newaxis]], axis=2)
    dirpath = '../dataset/day' + str(day)
    with open(os.path.join(dirpath, 'rain_matrix_mean.pickle'), 'wb') as f:
        pickle.dump(wind_matrix, f)